# Nearest Neighbour Imputation
#### Uses of the NNI technique in order to fill out the NaNs in the dataset

In [95]:
import pandas as pd
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import KNNImputer
from sklearn.impute import IterativeImputer
from numpy import isnan

### 1. Import the Dataset

In [97]:
# read csv datafile into pandas table
data = pd.read_csv("maps-synthetic-data-v1.1.csv")
pd.options.display.max_columns = None
data.head()
#display(data)

,Unnamed: 0,X,flag,comp_bed_9,mat_dep,mat_age,weight_16,height_16,iq,comp_noint_bed_16,comp_int_bed_16,talk_phon_wend,text_wend,talk_mob_wend,comp_wend,musi_wend,read_wend,work_wend,alon_wend,draw_wend,play_wend,tv_wend,out_win_wend,out_sum_wend,tran_wend,talk_phon_week,text_week,talk_mob_week,comp_week,musi_week,read_week,work_week,alon_week,draw_week,play_week,tv_week,out_win_week,out_sum_week,tran_week,pat_pres_10,pat_pres_8,pat_pres,num_home,mat_anx_1,mat_anx_18m,mat_anx_8m,agg_score,emot_cruel,phys_cruel,mat_anx_0m,pat_ses,mat_ses,pat_edu,mat_edu,parity,dep_band_15,dep_band_13,dep_band_10,dep_band_07,anx_band_15,anx_band_13,anx_band_10,anx_band_07,exercise,child_bull,phone_14_wend,phone_14_week,musi_13,tv_bed_9,own_mob,has_dep_diag,secd_diag,prim_diag,panic_score,dep_thoughts,dep_score,comp_house,tv_bed_16,creat_14,comp_games,fam_tv_eve,fam_tv_aft,fam_tv_mor,sex,birth_order
0,1,1,synthetic,NaN,3.0,30,59.294132,181.602831,107.0,NaN,NaN,Any at all,Less than 1 hour,Less than 1 hour,1-2 hours,Not at all,Less than 1 hour,3 or more hours,1-2 hours,Not at all,3 or more hours,1-2 hours,3 or more hours,3 or more hours,Less than 1 hour,Not at all,Less than 1 hour,Less than 1 hour,1-2 hours,Not at all,Less than 1 hour,3 or more hours,1-2 hours,Not at all,3 or more hours,1-2 hours,3 or more hours,3 or more hours,Less than 1 hour,Yes,Yes,Yes,4,No,No,No,11.0,No,No,No,III (manual),ii,A level,A level,0.0,~0.5%,~0.5%,~0.5%,~0.5%,~0.5%,~0.5%,~3%,~3%,5 or more times a week,No,NaN,NaN,NaN,NaN,NaN,No ICD-10 diagnosis of depression,0.0,0.0,0.0,NaN,0.0,Yes,Yes,often,No,"Yes, Some Days",Other,"Yes, Every Day",Female,A
1,2,2,synthetic,NaN,9.0,26,NaN,NaN,85.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,No,No,No,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Female,B
2,3,3,synthetic,NaN,3.0,24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,No,No,No,III (manual),iv,O level,O level,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Yes, Some Days","Yes, Every Day",Other,Male,A
3,4,4,synthetic,No,NaN,22,49.812426,160.224186,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,III (manual),i,O level,A level,0.0,NaN,NaN,<0.1%,NaN,NaN,NaN,~0.5%,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,Male,A
4,5,5,synthetic,No,8.0,31,62.270030,191.703227,132.0,Yes,NaN,Not at all,Not at all,Not at all,Less than 1 hour,Not at all,1 or more hours,1-2 hours,3 or more hours,Not at all,Less than 3 hours,3 or more hours,Less than 1 hour,Less than 3 hours,Less than 1 hour,Not at all,Not at all,Not at all,1-2 hours,Not at all,1 or more hours,1-2 hours,1-2 hours,Not at all,3 or more hours,1-2 hours,Less than 1 hour,Less than 3 hours,Less than 1 hour,NaN,NaN,No,4,No,No,No,9.0,Yes,No,No,III (manual),iv,A level,A level,1.0,<0.1%,<0.1%,<0.1%,NaN,~3%,~0.5%,~0.5%,NaN,1-4 times a week,No,Not at all,Not at all,No,Yes,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,often,No,"Yes, Some Days","Yes, Every Day","Yes, Every Day",Female,A


### 2. Exploring the Dataset

#### 2.1 Looking at the type of each column

In [102]:
data.info()
#data.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13734 entries, 0 to 13733
Data columns (total 85 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         13734 non-null  int64  
 1   X                  13734 non-null  int64  
 2   flag               13734 non-null  object 
 3   comp_bed_9         8174 non-null   object 
 4   mat_dep            11145 non-null  float64
 5   mat_age            13326 non-null  object 
 6   weight_16          5351 non-null   float64
 7   height_16          5364 non-null   float64
 8   iq                 7288 non-null   float64
 9   comp_noint_bed_16  471 non-null    object 
 10  comp_int_bed_16    2570 non-null   object 
 11  talk_phon_wend     4850 non-null   object 
 12  text_wend          4841 non-null   object 
 13  talk_mob_wend      4834 non-null   object 
 14  comp_wend          4839 non-null   object 
 15  musi_wend          4831 non-null   object 
 16  read_wend          483

#### 2.2 Exploring the Missing value of the dataset

In [103]:
data.isnull().sum()

Unnamed: 0        0
X                 0
flag              0
comp_bed_9     5560
mat_dep        2589
               ... 
fam_tv_eve     2560
fam_tv_aft     2696
fam_tv_mor     2638
sex               0
birth_order       0
Length: 85, dtype: int64

In [104]:
na_percentage = round(data.isnull().sum()/len(data) * 100, 2)
print(na_percentage)

na_pctlist = list(na_percentage)
na_col = list(data.columns)

# printing the whole column
for i in range(len(na_pctlist)):
    print(na_col[i] + ": " + str(na_pctlist[i]) + " %")

Unnamed: 0      0.00
X               0.00
flag            0.00
comp_bed_9     40.48
mat_dep        18.85
               ...  
fam_tv_eve     18.64
fam_tv_aft     19.63
fam_tv_mor     19.21
sex             0.00
birth_order     0.00
Length: 85, dtype: float64
Unnamed: 0: 0.0 %
X: 0.0 %
flag: 0.0 %
comp_bed_9: 40.48 %
mat_dep: 18.85 %
mat_age: 2.97 %
weight_16: 61.04 %
height_16: 60.94 %
iq: 46.93 %
comp_noint_bed_16: 96.57 %
comp_int_bed_16: 81.29 %
talk_phon_wend: 64.69 %
text_wend: 64.75 %
talk_mob_wend: 64.8 %
comp_wend: 64.77 %
musi_wend: 64.82 %
read_wend: 64.8 %
work_wend: 64.81 %
alon_wend: 64.69 %
draw_wend: 64.74 %
play_wend: 64.69 %
tv_wend: 64.66 %
out_win_wend: 64.69 %
out_sum_wend: 64.89 %
tran_wend: 64.61 %
talk_phon_week: 63.6 %
text_week: 63.56 %
talk_mob_week: 63.48 %
comp_week: 63.51 %
musi_week: 63.57 %
read_week: 63.64 %
work_week: 63.51 %
alon_week: 63.54 %
draw_week: 63.51 %
play_week: 63.48 %
tv_week: 63.45 %
out_win_week: 63.54 %
out_sum_week: 63.49 %
tran_week: 6

### 3. K-Nearest Neighbour Imputation

#### 3.1 Without the categorical data

In [105]:
# Choosing only the column which are numerical for testing purposes.
numr_data = [col for col in data.columns if data[col].dtypes != 'O']
data[numr_data].head()
data[numr_data].columns

Index(['Unnamed: 0', 'X', 'mat_dep', 'weight_16', 'height_16', 'iq',
       'agg_score', 'parity', 'secd_diag', 'prim_diag', 'panic_score',
       'dep_thoughts', 'dep_score'],
      dtype='object')

In [106]:
knn = KNNImputer(n_neighbors = 5, add_indicator = False)
knn.fit(data[numr_data])
knn.transform(data[numr_data])

knnpd1 = pd.DataFrame(knn.transform(data[numr_data]), columns=data[numr_data].columns)
knnpd1.head(20)

,Unnamed: 0,X,mat_dep,weight_16,height_16,iq,agg_score,parity,secd_diag,prim_diag,panic_score,dep_thoughts,dep_score
0,1.0,1.0,3.0,59.294132,181.602831,107.0,11.0,0.0,0.0,0.0,0.0,2.8,0.0
1,2.0,2.0,9.0,68.600018,174.780866,85.0,9.0,1.0,0.4,3.0,0.0,2.8,0.4
2,3.0,3.0,3.0,64.194092,175.856054,107.6,8.0,1.0,0.0,1.0,0.0,2.8,0.0
3,4.0,4.0,5.8,49.812426,160.224186,102.6,9.6,0.0,0.4,3.0,0.0,2.8,0.4
4,5.0,5.0,8.0,62.270030,191.703227,132.0,9.0,1.0,0.0,1.0,0.0,2.8,0.0
5,6.0,6.0,10.0,78.936613,169.722373,106.0,11.0,3.0,0.0,0.0,0.0,2.8,0.0
6,7.0,7.0,0.0,66.918576,180.412111,108.4,13.0,2.0,0.0,1.0,0.0,2.8,0.0
7,8.0,8.0,6.0,84.299663,186.327284,98.0,9.6,0.0,0.0,0.0,0.0,2.8,0.0
8,9.0,9.0,4.0,70.657258,176.027653,108.0,10.0,0.0,0.0,5.0,0.0,2.8,0.0
9,10.0,10.0,0.0,70.529403,187.778207,113.0,13.0,0.0,0.0,1.0,0.0,2.8,0.0


In [115]:
knn2 = KNNImputer(n_neighbors=3, add_indicator = False)
knn2.fit(data[numr_data])
knn2.transform(data[numr_data])

knnpd2 = pd.DataFrame(knn2.transform(data[numr_data]), columns=data[numr_data].columns)
knnpd2.head(20)

,Unnamed: 0,X,mat_dep,weight_16,height_16,iq,agg_score,parity,secd_diag,prim_diag,panic_score,dep_thoughts,dep_score
0,1.0,1.0,3.000000,59.294132,181.602831,107.000000,11.000000,0.0,0.000000,0.000000,0.0,2.666667,0.000000
1,2.0,2.0,9.000000,71.016234,172.091281,85.000000,9.000000,1.0,0.000000,0.000000,0.0,2.666667,0.000000
2,3.0,3.0,3.000000,57.125529,177.843415,108.000000,8.000000,1.0,0.000000,1.666667,0.0,2.666667,0.000000
3,4.0,4.0,4.000000,49.812426,160.224186,102.666667,10.000000,0.0,0.000000,1.666667,0.0,2.000000,0.000000
4,5.0,5.0,8.000000,62.270030,191.703227,132.000000,9.000000,1.0,0.000000,1.666667,0.0,2.333333,0.000000
5,6.0,6.0,10.000000,78.936613,169.722373,106.000000,11.000000,3.0,0.000000,0.000000,0.0,2.666667,0.000000
6,7.0,7.0,0.000000,68.213831,178.109893,106.333333,13.000000,2.0,0.000000,1.666667,0.0,2.666667,0.000000
7,8.0,8.0,3.000000,84.299663,186.327284,98.000000,10.333333,0.0,0.000000,0.000000,0.0,2.666667,0.000000
8,9.0,9.0,4.000000,70.657258,176.027653,108.000000,10.000000,0.0,0.000000,5.000000,0.0,2.666667,0.000000
9,10.0,10.0,0.000000,70.529403,187.778207,113.000000,13.000000,0.0,0.000000,1.666667,0.0,2.666667,0.000000


#### 3.2 Converting Categorical Data into Numberical Data

##### There are 2 types of categorical data:
1. Ordinal Relation
    - Each variable has a certain relationship, where if the order changes the meaning of the value will change.
    - For instance:
        - Shirt size ( S , M , L )
    * Approach is by using **Integer Encoding**
2. Non-Ordinal Relation
    - Each variable doesn't have any relationship, hence changing the order of the variable orientation wouldn't affect the meaning of the variable
    - For instance: 
        - Color ( R , G , B )
    * Approach is by using **One-Hot-Encoding**

In [107]:
# read csv datafile into pandas table
cat2num = pd.read_csv("KNNI_cat2num.csv")
pd.options.display.max_columns = None
cat2num.head()

,Unnamed: 0,X,comp_bed_9,mat_dep,mat_age,weight_16,height_16,iq,comp_noint_bed_16,comp_int_bed_16,talk_phon_wend,text_wend,talk_mob_wend,comp_wend,musi_wend,read_wend,work_wend,alon_wend,draw_wend,play_wend,tv_wend,out_win_wend,out_sum_wend,tran_wend,talk_phon_week,text_week,talk_mob_week,comp_week,musi_week,read_week,work_week,alon_week,draw_week,play_week,tv_week,out_win_week,out_sum_week,tran_week,pat_pres_10,pat_pres_8,pat_pres,num_home,mat_anx_1,mat_anx_18m,mat_anx_8m,agg_score,emot_cruel,phys_cruel,mat_anx_0m,pat_ses,mat_ses,pat_edu,mat_edu,parity,dep_band_15,dep_band_13,dep_band_10,dep_band_07,anx_band_15,anx_band_13,anx_band_10,anx_band_07,exercise,child_bull,phone_14_wend,phone_14_week,musi_13,tv_bed_9,own_mob,has_dep_diag,secd_diag,prim_diag,panic_score,dep_thoughts,dep_score,comp_house,tv_bed_16,creat_14,comp_games,fam_tv_eve,fam_tv_aft,fam_tv_mor,sex,birth_order
0,1,1,NaN,3.0,30.0,59.294132,181.602831,107.0,NaN,NaN,1.0,1.0,1.0,2.0,0.0,1.0,3.0,2.0,0.0,1.0,2.0,3.0,1.0,0.0,0.0,1.0,1.0,2.0,0.0,1.0,3.0,2.0,0.0,1.0,2.0,3.0,1.0,0.0,1.0,1.0,1.0,4,0.0,0.0,0.0,11.0,0.0,0.0,0.0,3.0,2.0,3.0,3.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,4.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,0.0,1.0,1.0,1.0,0.0,0.0,2.0,1.0,1,0
1,2,2,NaN,9.0,26.0,NaN,NaN,85.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
2,3,3,NaN,3.0,24.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,0.0,0.0,0.0,3.0,5.0,2.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,2.0,0,0
3,4,4,0.0,NaN,22.0,49.812426,160.224186,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,1.0,2.0,3.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0,0
4,5,5,0.0,8.0,31.0,62.270030,191.703227,132.0,1.0,NaN,0.0,0.0,0.0,1.0,0.0,2.0,2.0,3.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,2.0,2.0,0.0,1.0,2.0,1.0,0.0,0.0,NaN,NaN,0.0,4,0.0,0.0,0.0,9.0,1.0,0.0,0.0,3.0,5.0,3.0,3.0,1.0,0.0,0.0,0.0,NaN,2.0,1.0,1.0,NaN,3.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,1.0,1.0,1,0


In [108]:
tonum = [col for col in cat2num.columns if cat2num[col].dtypes != 'O']
#df[tonum].head()
cat2num[tonum].columns
#cat2num[tonum].shape[1]

Index(['Unnamed: 0', 'X', 'comp_bed_9', 'mat_dep', 'mat_age', 'weight_16',
       'height_16', 'iq', 'comp_noint_bed_16', 'comp_int_bed_16',
       'talk_phon_wend', 'text_wend', 'talk_mob_wend', 'comp_wend',
       'musi_wend', 'read_wend', 'work_wend', 'alon_wend', 'draw_wend',
       'play_wend', 'tv_wend', 'out_win_wend', 'out_sum_wend', 'tran_wend',
       'talk_phon_week', 'text_week', 'talk_mob_week', 'comp_week',
       'musi_week', 'read_week', 'work_week', 'alon_week', 'draw_week',
       'play_week', 'tv_week', 'out_win_week', 'out_sum_week', 'tran_week',
       'pat_pres_10', 'pat_pres_8', 'pat_pres', 'mat_anx_1', 'mat_anx_18m',
       'mat_anx_8m', 'agg_score', 'emot_cruel', 'phys_cruel', 'mat_anx_0m',
       'pat_ses', 'mat_ses', 'pat_edu', 'mat_edu', 'parity', 'dep_band_15',
       'dep_band_13', 'dep_band_10', 'dep_band_07', 'anx_band_15',
       'anx_band_13', 'anx_band_10', 'anx_band_07', 'exercise', 'child_bull',
       'phone_14_wend', 'phone_14_week', 'musi_13', 't

In [113]:
knn_allnum = KNNImputer(n_neighbors = 5, add_indicator = False)
knn_allnum.fit(cat2num[tonum])
knn_allnum.transform(cat2num[tonum])

knn_cat2num = pd.DataFrame(knn_allnum.transform(cat2num[tonum]), columns=cat2num[tonum].columns)
knn_cat2num.head(20)

,Unnamed: 0,X,comp_bed_9,mat_dep,mat_age,weight_16,height_16,iq,comp_noint_bed_16,comp_int_bed_16,talk_phon_wend,text_wend,talk_mob_wend,comp_wend,musi_wend,read_wend,work_wend,alon_wend,draw_wend,play_wend,tv_wend,out_win_wend,out_sum_wend,tran_wend,talk_phon_week,text_week,talk_mob_week,comp_week,musi_week,read_week,work_week,alon_week,draw_week,play_week,tv_week,out_win_week,out_sum_week,tran_week,pat_pres_10,pat_pres_8,pat_pres,mat_anx_1,mat_anx_18m,mat_anx_8m,agg_score,emot_cruel,phys_cruel,mat_anx_0m,pat_ses,mat_ses,pat_edu,mat_edu,parity,dep_band_15,dep_band_13,dep_band_10,dep_band_07,anx_band_15,anx_band_13,anx_band_10,anx_band_07,exercise,child_bull,phone_14_wend,phone_14_week,musi_13,tv_bed_9,own_mob,has_dep_diag,secd_diag,prim_diag,panic_score,dep_thoughts,dep_score,comp_house,tv_bed_16,creat_14,comp_games,fam_tv_eve,fam_tv_aft,fam_tv_mor,sex,birth_order
0,1.0,1.0,0.0,3.0,30.0,59.294132,181.602831,107.0,1.0,1.0,1.0,1.0,1.0,2.0,0.0,1.0,3.0,2.0,0.0,1.0,2.0,3.0,1.0,0.0,0.0,1.0,1.0,2.0,0.0,1.0,3.0,2.0,0.0,1.0,2.0,3.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,11.0,0.0,0.0,0.0,3.0,2.0,3.0,3.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,4.0,0.0,0.4,0.6,0.6,0.4,0.2,0.0,0.0,0.0,0.0,2.6,0.0,1.0,1.0,1.0,0.0,0.0,2.0,1.0,1.0,0.0
1,2.0,2.0,0.0,9.0,26.0,68.600018,174.780866,85.0,1.0,1.0,0.8,0.8,0.8,1.8,0.4,0.8,2.2,2.6,0.2,0.8,2.4,2.4,0.8,0.2,0.6,1.2,1.0,2.0,0.4,0.8,2.6,1.8,0.2,1.0,1.8,2.4,0.8,0.4,0.6,0.6,0.6,0.2,0.2,0.2,9.0,0.0,0.0,0.0,3.2,3.0,2.4,1.6,1.0,1.0,0.6,0.0,1.0,1.6,1.8,1.8,1.6,3.2,0.0,1.2,1.2,0.6,0.4,0.2,0.2,0.4,3.0,0.0,2.8,0.4,1.0,1.0,0.2,0.6,0.4,1.2,1.8,1.0,1.0
2,3.0,3.0,0.0,3.0,24.0,66.922573,177.915980,105.6,1.0,1.0,0.6,0.6,0.6,1.8,0.2,1.0,2.4,2.6,0.2,0.6,2.6,2.4,0.8,0.2,0.4,1.0,0.8,2.4,0.2,1.0,2.8,2.2,0.2,0.8,2.2,2.2,0.8,0.4,0.6,0.8,0.4,0.2,0.0,0.0,8.0,0.0,0.0,0.0,3.0,5.0,2.0,2.0,1.0,0.8,0.6,0.2,0.4,1.8,1.4,1.4,1.4,3.2,0.0,0.6,0.6,0.6,0.4,0.2,0.0,0.0,1.0,0.0,2.8,0.0,1.0,1.0,0.6,0.4,0.0,1.0,2.0,0.0,0.0
3,4.0,4.0,0.0,5.8,22.0,49.812426,160.224186,98.6,1.0,1.0,0.8,0.8,0.8,2.0,0.4,0.6,2.4,2.6,0.2,0.8,2.4,2.8,1.0,0.2,0.6,1.2,1.0,2.2,0.4,0.6,2.8,2.2,0.2,0.8,2.0,2.4,1.0,0.4,0.8,1.0,0.8,0.0,0.2,0.2,9.6,0.0,0.0,0.2,3.0,1.0,2.0,3.0,0.0,0.8,0.4,0.0,1.0,1.6,1.6,1.0,1.8,3.2,0.0,1.2,1.0,0.8,0.0,0.4,0.0,0.0,1.0,0.0,2.8,0.0,1.0,1.0,0.4,0.0,0.2,1.2,1.8,0.0,0.0
4,5.0,5.0,0.0,8.0,31.0,62.270030,191.703227,132.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,2.0,2.0,3.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,2.0,2.0,0.0,1.0,2.0,1.0,0.0,0.0,0.8,0.8,0.0,0.0,0.0,0.0,9.0,1.0,0.0,0.0,3.0,5.0,3.0,3.0,1.0,0.0,0.0,0.0,1.0,2.0,1.0,1.0,1.6,3.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,2.8,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0
5,6.0,6.0,0.0,10.0,30.0,78.936613,169.722373,106.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,3.0,0.0,1.0,3.0,2.0,1.0,0.0,1.0,1.0,1.0,2.0,0.0,1.0,3.0,2.2,0.0,1.0,2.0,3.0,1.0,1.0,1.0,1.0,0.8,0.0,0.2,0.2,11.0,0.0,0.0,0.0,4.0,5.0,1.0,1.0,3.0,1.0,2.0,0.0,0.0,1.0,3.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.6,0.0,1.0,1.0,0.0,1.0,0.0,2.0,0.0,1.0,0.0
6,7.0,7.0,0.0,0.0,36.0,69.410097,184.687840,111.6,1.0,1.0,0.6,0.6,0.6,1.8,0.2,1.0,2.4,2.6,0.2,0.6,2.6,2.4,0.8,0.2,0.4,1.0,0.8,2.4,0.2,1.0,2.8,2.2,0.2,0.8,2.2,2.2,0.8,0.4,0.6,0.6,1.0,0.0,0.0,0.0,13.0,0.0,0.0,0.0,2.0,4.0,2.0,1.0,2.0,0.8,0.2,0.2,0.6,1.8,1.2,1.4,1.8,3.2,0.0,0.4,0.6,0.6,0.6,0.2,0.0,0.0,1.0,0.0,2.8,0.0,1.0,1.0,0.6,1.0,1.0,1.0,2.0,0.0,0.0
7,8.0,8.0,0.0,6.4,26.0,84.299663,186.327284,98.0,1.0,1.0,0.8,0.8,0.8,1.8,0.4,0.8,2.2,2.6,0.2,0.8,2.4,2.4,0.8,0.2,0.6,1.2,1.0,2.0,0.4,0.8,2.6,1.8,0.2,1.0,1.8,2.4,0.8,0.4,0.0,0.0,0.0,1.0,0.2,0.0,9.6,0.0,0.0,0.4,3.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,1.0,1.0,3.2,0.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,2.8,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0
8,9.0,9.0,0.0,4.0,30.0,70.657258,176.027653,108.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,0.0,3.0,2.0,1.0,1.0,2.0,3.0,1.0,1.0,1.0,3.0,2.0,3.0,1.0,0.0,3.0,2.0,1.0,1.0,2.0,3.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,2.0,4.0,3.0,1.0,0.0,2.0,0.0,0.0,0.0,3.0,1.0,1.0,1.0,4.0,0.0,0.8,0.8,0.8,0.0,0.0,0.0,0.0,5.0,0.0,2.6,0.0,1.0,1.